# Integrating LangChain Agents with Claude and Bedrock

In this notebook, we integrate 3 external tools using LangChain Agent tools with Anthropic Claude as our LLM

- Wikipedia (to answer general questions about people, history etc)
- DuckduckGo search (to answer questions on current events)
- A custom calculator tool to perform calculations

In [1]:
# ! pip install wikipedia duckduckgo-search --quiet

In [2]:
import json
from pathlib import Path

import boto3
from langchain.embeddings.bedrock import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from rich import print
from utils import get_inference_parameters, save_prompt

%load_ext rich
%load_ext autoreload
%autoreload 2

### Initialize LLM

We use `claude-instant` model for quicker responses. You may switch to `claude-v2` for larger context lengths.

In [3]:
region = "us-west-2"
bedrock_client = boto3.client("bedrock-runtime", region_name=region)
# model_id = "anthropic.claude-v2"
model_id = "anthropic.claude-instant-v1"
model_kwargs = get_inference_parameters("anthropic")

# Instantial LLM
llm = Bedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=model_kwargs,
    region_name=region,
)

In [4]:
print(model_kwargs)

{'max_tokens_to_sample': 1000, 'temperature': 0.0, 'top_k': 250, 'top_p': 0.999, 'stop_sequences': ['\n\nHuman:']}

### Initialize tools

- Wikipedia
- Duckduckgo Search
- Custom Math Tool

In [5]:
from langchain.agents import Tool
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities import WikipediaAPIWrapper

# Define the API wrapper for Wikipedia
wikipedia = WikipediaAPIWrapper()

# Define the Wikipedia tool with a description and the function to retrieve results from Wikipedia
wikipedia_tool = Tool(
    name="wikipedia",
    func=wikipedia.run,
    description="Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.",
)

# Define the API wrapper for DuckDuckGo search
duckduckgo_search = DuckDuckGoSearchRun()

# Define the DuckDuckGo tool using a description and the function to retrieve results from DuckDuckGo search
duckduckgo_tool = Tool(
    name="DuckDuckGoSearch",
    func=duckduckgo_search.run,
    description="useful for when you need to answer questions about current events",
)

#### Custom Math tool

Here, we create a custom `math_chain` with custom `model_kwargs` and define this as a tool to be used by the LLM.

In [6]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.prompts.prompt import PromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

# Math Prompt Template
math_template = """Human: Given a question with a math problem, provide only a single line mathematical expression that solves the problem in the following format. Don't solve the expression only create a parsable expression.
```text
${{single line mathematical expression that solves the problem}}
```

Assistant:
 Here is an example response with a single line mathematical expression for solving a math problem:
```text
37593**(1/5)
```

Human: {question}

Assistant:"""

math_prompt = PromptTemplate.from_template(template=math_template)

# Create a new LLM connection with custom `stop_sequence`
math_model_kwargs = model_kwargs
# Update the stop_seqeunce based on prompt_template
math_model_kwargs["stop_sequences"] = ["```output"]

math_llm = Bedrock(
    client=bedrock_client,
    model_id=model_id,
    model_kwargs=math_model_kwargs,
    region_name=region,
)

# Create a custom Math chain that'll use LLM to determine the output
math_chain = {"question": RunnablePassthrough()} | math_prompt | math_llm | StrOutputParser()

# Define the tool
math_tool = Tool.from_function(
    func=math_chain.invoke,
    name="Calculator",
    description="Useful for when you need to answer questions about math.",
)

print(math_chain)

RunnableSequence(
    first=RunnableParallel(steps={'question': RunnablePassthrough()}),
    middle=[
        PromptTemplate(
            input_variables=['question'],
            template="Human: Given a question with a math problem, provide only a single line mathematical 
expression that solves the problem in the following format. Don't solve the expression only create a parsable 
expression.\n```text\n${{single line mathematical expression that solves the problem}}\n```\n\nAssistant:\n Here is
an example response with a single line mathematical expression for solving a math 
problem:\n```text\n37593**(1/5)\n```\n\nHuman: {question}\n\nAssistant:"
        ),
        Bedrock(
            client=<botocore.client.BedrockRuntime object at 0x10da0e850>,
            region_name='us-west-2',
            model_id='anthropic.claude-instant-v1',
            model_kwargs={
                'max_tokens_to_sample': 1000,
                'temperature': 0.0,
                'top_k': 250,
                'top_p': 0.999,
                'stop_sequences': ['```output']
            }
        )
    ],
    last=StrOutputParser()
)

### Prepare ReAct prompt for Agent

For the Agent to utilize the tools defined in the prompt, we define a ReAct (**Re**ason and **Act**) prompt template that tells which tools are avialable and how to work through the question.

First, we define a list of tools and then create text strings that define:
1. `tools_string`: Formats text in `tool_name: tool_description \n` format
2. `tool_names` : Collect the list of tool names as defined in the tool definitions above.

For simplicity, we break the prompt into 3 parts:
- *Prefix:* prefix the prompt with `HUMAN_PROMPT` and fill with `tools_string`
- *ReAct:* This portion of the prompt will replace `tool_names` to be used in the template
- *Suffix:* Finally, we add the Question in xml tags and Assistant tags
- **Final Prompt:** We concatenate all 3 portions above to form the final prompt

*Optional:* Uncomment the code at the end of the cell save the prompt to disk as text file

In [7]:
# construct list of tools for the agent
tools = [wikipedia_tool, duckduckgo_tool, math_tool]

# Prepare tools_string with `tool name: tool description` to inject into the prompt

tools_string = ""
tool_names = []  # collect tool names
for tool in tools:
    if isinstance(tool, Tool):
        tool_names.append(tool.name)
        tools_string += f"{tool.name}: {tool.description}\n"

# Optional: shuffle the tool_names list
# import random
# random.shuffle(tool_names)

In [8]:
from anthropic_bedrock import HUMAN_PROMPT

# Before initializing the agent we need to provide anthropic formatter prompt that tells the tools to use and how to use it
prompt_prefix = """You are helpful AI research assistant. Your task is to answer the user's question in <question> tags in a truthful manner.

You have access to the following tools:
{tools_string}
"""

prompt_react = """Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of {tool_names}
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question in <answer> tags

"""
# Replace tool_string
prompt_prefix = f"{HUMAN_PROMPT}{prompt_prefix.format(tools_string=tools_string)}"
# Replace tool_names
prompt_react = prompt_react.format(tool_names=tool_names)

# Tool names are hard
prompt_suffix = """Question: <question> {input} </question>

Assistant:
{agent_scratchpad}"""

agent_prompt_template = f"{prompt_prefix}{prompt_react}{prompt_suffix}"

# print(agent_prompt_template)
# Optional: Save prompt to disk
# prompt_save_path = Path('./prompts/react_agent_prompt_anthropic.txt')
# print(f"Saving prompt to: {str(prompt_save_path)}")
# save_prompt(agent_prompt_template, prompt_save_path)

### Initialize Agent 

We pass in the list of tools, llm and `AgentType` to initialize the agent. We also assign the ReAct prompt to the llm used by the Agent.

In [9]:
# Initialize the agent with access to all the tools in the list
agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

# assign the prompt to the agent
agent_executor.agent.llm_chain.prompt.template = agent_prompt_template

### Execute (run) the agent with an input

Extract the answer from `<answer>` tags from output.

In [10]:
import re

# question = "What is Mixture of Experts in Machine Learning"  # Test wikipedia
# question = "What is FlashAttention? What are it's benefits"
# question = "What is Mixtral and how is it different from other Mistral models"
question = "How does Mixtral perform comparing to other SOTA models"  # Test web search
# question = "What is conciousness"
# question = "Get me the biography of Chester Bennington. Include his birth date, city and country"
# question = "Amazon SageMaker launch year multiplied by 2"  # Test math_tool

output = agent_executor.run(question)

# extract answer from <answer> tags
final_output = re.findall(r"<answer>(.*?)</answer>", output, flags=re.DOTALL)[0]
print(final_output)



> Entering new AgentExecutor chain...
 Here is my attempt at answering the question:

Question: <question> How does Mixtral perform comparing to other SOTA models </question>

Thought: I don't have direct knowledge of how Mixtral compares to other models. Let me search online for information.

Action: Wikipedia 
Action Input: Mixtral

Observation: Wikipedia is not a valid tool, try one of [wikipedia, DuckDuckGoSearch, Calculator].
Thought: Let me try searching online instead.

Action: DuckDuckGoSearch  
Action Input: Mixtral model performance compared to other SOTA models

Observation: To that end, Azure AI Studio provides a model benchmarking and evaluation subsystem, which is an invaluable tool for users to review and compare the performance of various AI models. The platform provides quality metrics for Azure OpenAI Service models and Llama 2 models such as Llama-2-7b, gpt-4, gpt-4-32k, and gpt-35-turbo. Mixtral 8x7b is an exciting new LLM released by Mistral, which sets a new sta

Based on the search results, Mixtral is a new large language model released by Mistral AI that outperforms other 
SOTA models like GPT-3.5 and Llama 2 on many benchmarks while being faster and more efficient. It uses a novel 
sparse mixture-of-experts architecture.